In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns


In [ ]:
dataset/
    train/
        class1/
        class2/
    val/
        class1/
        class2/
    test/
        class1/
        class2/


In [ ]:
#Option A — If dataset ALREADY has train / val / test folders
data_dir = "/content/dataset"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir + "/train",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir + "/val",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

test_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir + "/test",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)


In [ ]:
#Option B — If dataset has ONLY one folder → auto-split
data_dir = "/content/dataset"

train_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    validation_split=0.2,
    subset="training",
    seed=42
)

val_ds = keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    validation_split=0.2,
    subset="validation",
    seed=42
)


In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

preprocess_layer = layers.Rescaling(1/255)


In [ ]:
model = keras.Sequential([
    layers.Input(shape=(224, 224, 3)),

    data_augmentation,
    preprocess_layer,

    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation="relu"),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(train_ds.cardinality().numpy(), activation="softmax")
])


In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
plt.figure(figsize=(12,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy")
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Loss")
plt.legend()

plt.show()


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)


In [ ]:
# Get true labels
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# Get predictions
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

cm = confusion_matrix(y_true, y_pred)
print(cm)


In [ ]:
from tensorflow.keras.preprocessing import image

img_path = "/content/sample.jpg"
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, 0)  # create batch axis
img_array = img_array / 255.0

pred = model.predict(img_array)
class_index = pred.argmax()
print("Predicted class:", train_ds.class_names[class_index])
